In [14]:
!git pull https://github.com/Lolleeee/MIEEG

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 3), reused 6 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 7.25 KiB | 3.62 MiB/s, done.
From https://github.com/Lolleeee/MIEEG
 * branch            HEAD       -> FETCH_HEAD
Updating ce1c366..1d47c60
Fast-forward
 kaggle_api.ipynb                    | 190 +++++++++++++++
 packages/models/autoencoder_skip.py | 466 ++++++++++++++++++++++++++++++++++++
 2 files changed, 656 insertions(+)
 create mode 100644 kaggle_api.ipynb
 create mode 100644 packages/models/autoencoder_skip.py


In [5]:
!git clone https://github.com/Lolleeee/MIEEG
import os 
os.chdir("MIEEG")

Cloning into 'MIEEG'...
remote: Enumerating objects: 899, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 899 (delta 27), reused 35 (delta 23), pack-reused 849 (from 1)
Receiving objects: 100% (899/899), 38.42 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (579/579), done.


In [6]:
import numpy as np
matrix = np.array(
    [
        [None, "Fp1", None, "Fp2", None],
        ["F7", "F3", "Fz", "F4", "F8"],
        ["FC5", "FC1", "Cz", "FC2", "FC6"],
        ["T7", "C3", "CP1", "C4", "T8"],
        ["TP9", "CP5", "CP2", "CP6", "TP10"],
        ["P7", "P3", "Pz", "P4", "P8"],
        ["PO9", "O1", "Oz", "O2", "PO10"],
    ]
)

In [ ]:
from packages.models.autoencoder_skip import Conv3DAE 
model = Conv3DAE(
        in_channels=25, 
        input_spatial=(7, 5, 64),  
        latent_dim=512, 
        hidden_dims=[128, 256],
        use_convnext=True,
        convnext_expansion=4,
        drop_p=0.1)


In [11]:
from packages.train.training import train_model
from packages.train.loss import CustomMSELoss, PerceptualLoss, CustomL1Loss
from packages.io.file_loader import get_data_loaders
import torch
import os

from packages.data_objects.dataset import TorchDataset

dataset_path = "/kaggle/input/64nooverlap05"
# Dummy training loop
optimizer = torch.optim.AdamW

mae = torch.nn.L1Loss
criterion = CustomMSELoss(scale=1, masked=True, matrix=matrix )
config = {
    'weight_decay': 1e-4,
    'epochs': 50,
    'EarlyStopping' : {'patience': 20, 'min_delta': 0.01},
    'BackupManager': {'backup_interval': 10, 'backup_path': './model_backups'},
    'ReduceLROnPlateau': {'mode': 'min', 'patience': 5, 'factor': 0.0},
    'history_plot': {'plot_type': 'extended', 'save_path': './training_history'},
    'grad_clip': 1.0,
    'use_amp': False,
    'grad_logging_interval' : None,
    'asym_lr': None
}

dataset = TorchDataset(root_folder=dataset_path)

train_loader, val_loader, _ = get_data_loaders(dataset, sets_size={'train': 0.7, 'val': 0.3, 'test': 0.}, norm_axes=(0, 4), batch_size = 64)

Calculating global parameters: 100%|██████████| 338/338 [00:16<00:00, 20.33it/s]


In [13]:
model = train_model(model, train_loader=train_loader, val_loader=val_loader, loss_criterion=criterion, optimizer=optimizer, config=config, metrics={})

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/338 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
from packages.train.testing import autoencoder_test_plots
autoencoder_test_plots(model, val_loader, nsamples = 5)

RuntimeError: Expected weight to be a vector of size equal to the number of channels in input, but got weight of shape [128] and input of shape [128, 7, 5, 64]